### Alter and update EMPIRE Input - Sets.xlsx

In [2]:
import pandas as pd
import numpy as np

In [3]:
NODES_TO_BE_REMOVED = ["Utsira Nord", "Sørlige Nordsjø I", "Sørlige Nordsjø II"]

#### Tab: Nodes

In [4]:
nodes_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="Nodes")
nodes_tab

Node
0              Austria
1             Bosnia H
2              Belgium
3             Bulgaria
4          Switzerland
5              Czech R
6              Germany
7              Denmark
8              Estonia
9                Spain
10             Finland
11              France
12         Great Brit.
13              Greece
14             Croatia
15             Hungary
16             Ireland
17               Italy
18           Lithuania
19             Luxemb.
20              Latvia
21           Macedonia
22         Netherlands
23                 NO1
24                 NO2
25                 NO3
26                 NO4
27                 NO5
28              Poland
29            Portugal
30             Romania
31              Serbia
32              Sweden
33            Slovenia
34            Slovakia
35         Moray Firth
36      Firth of Forth
37         Dogger Bank
38             Hornsea
39       Outer Dowsing
40             Norfolk
41         East Anglia
42            Borssele
43     Hollandsee Kust
44   Helgoländer Bucht
45            Nordsøen
46         Utsira Nord
47   Sørlige Nordsjø I
48  Sørlige Nordsjø II
49        Energyhub EU

Update with new OW nodes

In [5]:
OW_nodes = pd.read_csv("Output/OW_nodes.csv")
OW_nodes = OW_nodes[["Node"]]
nodes_tab = pd.concat([nodes_tab[~nodes_tab["Node"].isin(NODES_TO_BE_REMOVED)], OW_nodes]).drop_duplicates().reset_index(drop=True)
nodes_tab

Node
0       Austria
1      Bosnia H
2       Belgium
3      Bulgaria
4   Switzerland
..          ...
62  Nordavind A
63  Nordavind D
64  Nordavind C
65  Vestavind E
66    Sørvest D

[67 rows x 1 columns]

#### Tab: Storage

Keep as is

#### Tab: Technology 

Keep as is

#### Tab: Generators

Keep as is

#### Tab: LineType

Keep as is

#### Tab: StorageOfNodes

Keep as is

#### Tab: DirectionalLines

In [6]:
directional_lines_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="DirectionalLines", header=2)[["NodeFrom", "NodeTo"]]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeFrom"].isin(NODES_TO_BE_REMOVED)]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeTo"].isin(NODES_TO_BE_REMOVED)].reset_index(drop=True)
directional_lines_tab

NodeFrom        NodeTo
0    Switzerland       Austria
1        Czech R       Austria
2        Germany       Austria
3        Hungary       Austria
4          Italy       Austria
..           ...           ...
339  Great Brit.  Energyhub EU
340  Netherlands  Energyhub EU
341      Germany  Energyhub EU
342      Denmark  Energyhub EU
343      Belgium  Energyhub EU

[344 rows x 2 columns]

Grab newly generated directional lines

In [7]:
df_dl = pd.read_csv("Output/directional_lines.csv")
df_dl

FromNode          ToNode  lineLength in km                LineType
0       Moray Firth  Firth of Forth        215.667240              HVDC_Cable
1    Firth of Forth     Dogger Bank        333.632410              HVDC_Cable
2    Firth of Forth         Hornsea        387.595780              HVDC_Cable
3       Dogger Bank         Hornsea        120.257880              HVDC_Cable
4    Firth of Forth   Outer Dowsing        400.736837              HVDC_Cable
..              ...             ...               ...                     ...
230     Nordavind A             NO4         82.890254  HVDC_OffshoreToOnshore
231     Nordavind D             NO4        135.057542  HVDC_OffshoreToOnshore
232     Nordavind C             NO4        139.913997  HVDC_OffshoreToOnshore
233     Vestavind E             NO2         61.196428  HVDC_OffshoreToOnshore
234       Sørvest D             NO2        250.690470  HVDC_OffshoreToOnshore

[235 rows x 4 columns]

In [8]:
offshr_lines = df_dl[df_dl["LineType"] == "HVDC_Cable"]
offshr_areas = set(offshr_lines["FromNode"].values).union(offshr_lines["ToNode"].values)
offshr_areas

{'Borssele',
 'Dogger Bank',
 'East Anglia',
 'Energyhub EU',
 'Firth of Forth',
 'Helgoländer Bucht',
 'Hollandsee Kust',
 'Hornsea',
 'Moray Firth',
 'Nordavind A',
 'Nordavind B',
 'Nordavind C',
 'Nordavind D',
 'Nordsøen',
 'Nordvest A',
 'Nordvest B',
 'Nordvest C',
 'Norfolk',
 'Outer Dowsing',
 'Sønnavind A',
 'Sørvest A',
 'Sørvest B',
 'Sørvest C',
 'Sørvest D',
 'Sørvest E',
 'Sørvest F',
 'Vestavind A',
 'Vestavind B',
 'Vestavind C',
 'Vestavind D',
 'Vestavind E',
 'Vestavind F'}

Remove cables in original set that are to be replaced 

In [9]:
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeFrom"].isin(offshr_areas)]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeTo"].isin(offshr_areas)].reset_index(drop=True)
directional_lines_tab

NodeFrom       NodeTo
0    Switzerland      Austria
1        Czech R      Austria
2        Germany      Austria
3        Hungary      Austria
4          Italy      Austria
..           ...          ...
165        Italy     Slovenia
166      Austria  Switzerland
167      Czech R     Slovakia
168      Hungary     Slovakia
169       Poland     Slovakia

[170 rows x 2 columns]

Add newly generated lines (two-way conns)

In [10]:
first_way = df_dl[["FromNode", "ToNode"]].rename(columns={"FromNode": "NodeFrom", "ToNode": "NodeTo"})
second_way = first_way.copy().rename(columns={"NodeFrom": "NodeTo", "NodeTo": "NodeFrom"})
second_way

NodeTo        NodeFrom
0       Moray Firth  Firth of Forth
1    Firth of Forth     Dogger Bank
2    Firth of Forth         Hornsea
3       Dogger Bank         Hornsea
4    Firth of Forth   Outer Dowsing
..              ...             ...
230     Nordavind A             NO4
231     Nordavind D             NO4
232     Nordavind C             NO4
233     Vestavind E             NO2
234       Sørvest D             NO2

[235 rows x 2 columns]

In [11]:
directional_lines_tab = pd.concat([directional_lines_tab, first_way, second_way], ignore_index=True).reset_index(drop=True)
directional_lines_tab

NodeFrom       NodeTo
0    Switzerland      Austria
1        Czech R      Austria
2        Germany      Austria
3        Hungary      Austria
4          Italy      Austria
..           ...          ...
635          NO4  Nordavind A
636          NO4  Nordavind D
637          NO4  Nordavind C
638          NO2  Vestavind E
639          NO2    Sørvest D

[640 rows x 2 columns]

#### Tab: LineTypeOfDirectionalLines

Same structure as Tab: DirectionalLines but with LineType incl. 

In [12]:
line_type_of_directional_lines_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="LineTypeOfDirectionalLines", header=2)
line_type_of_directional_lines_tab

FromNode        ToNode                LineType
0    Switzerland       Austria       HVAC_OverheadLine
1        Czech R       Austria       HVAC_OverheadLine
2        Germany       Austria       HVAC_OverheadLine
3        Hungary       Austria       HVAC_OverheadLine
4          Italy       Austria       HVAC_OverheadLine
..           ...           ...                     ...
419          NO2  Energyhub EU  HVDC_OffshoreToOnshore
420  Great Brit.  Energyhub EU  HVDC_OffshoreToOnshore
421  Netherlands  Energyhub EU  HVDC_OffshoreToOnshore
422      Germany  Energyhub EU  HVDC_OffshoreToOnshore
423      Denmark  Energyhub EU  HVDC_OffshoreToOnshore

[424 rows x 3 columns]

In [13]:
# Output format with same LineType as before on non-touched lines
df_ltodl = directional_lines_tab.copy().rename(columns={"NodeFrom": "FromNode", "NodeTo": "ToNode"})
df_ltodl = df_ltodl.merge(line_type_of_directional_lines_tab, on=["FromNode", "ToNode"], how="left")
df_ltodl

FromNode       ToNode           LineType
0    Switzerland      Austria  HVAC_OverheadLine
1        Czech R      Austria  HVAC_OverheadLine
2        Germany      Austria  HVAC_OverheadLine
3        Hungary      Austria  HVAC_OverheadLine
4          Italy      Austria  HVAC_OverheadLine
..           ...          ...                ...
635          NO4  Nordavind A                NaN
636          NO4  Nordavind D                NaN
637          NO4  Nordavind C                NaN
638          NO2  Vestavind E                NaN
639          NO2    Sørvest D                NaN

[640 rows x 3 columns]

In [14]:
df_ltodl.isna().sum()

FromNode      0
ToNode        0
LineType    344
dtype: int64

In [15]:
# LineType of others can be found using df_dl but need to map two-ways

# First-way
offshr_areas_lines = df_dl.copy().drop(columns=["lineLength in km"])
mapping = offshr_areas_lines.set_index(['FromNode', 'ToNode'])['LineType']
df_ltodl['LineType'] = df_ltodl.apply(lambda row: mapping.get((row['FromNode'], row['ToNode']), row['LineType']), axis=1)

# Second-way
offshr_areas_lines = df_dl.copy().drop(columns=["lineLength in km"]).rename(columns={"FromNode": "ToNode", "ToNode": "FromNode"})
mapping = offshr_areas_lines.set_index(['FromNode', 'ToNode'])['LineType']
df_ltodl['LineType'] = df_ltodl.apply(lambda row: mapping.get((row['FromNode'], row['ToNode']), row['LineType']), axis=1)
df_ltodl

FromNode       ToNode                LineType
0    Switzerland      Austria       HVAC_OverheadLine
1        Czech R      Austria       HVAC_OverheadLine
2        Germany      Austria       HVAC_OverheadLine
3        Hungary      Austria       HVAC_OverheadLine
4          Italy      Austria       HVAC_OverheadLine
..           ...          ...                     ...
635          NO4  Nordavind A  HVDC_OffshoreToOnshore
636          NO4  Nordavind D  HVDC_OffshoreToOnshore
637          NO4  Nordavind C  HVDC_OffshoreToOnshore
638          NO2  Vestavind E  HVDC_OffshoreToOnshore
639          NO2    Sørvest D  HVDC_OffshoreToOnshore

[640 rows x 3 columns]

In [16]:
df_ltodl.isna().sum()

FromNode    0
ToNode      0
LineType    0
dtype: int64

In [17]:
line_type_of_directional_lines_tab = df_ltodl.copy()
line_type_of_directional_lines_tab

FromNode       ToNode                LineType
0    Switzerland      Austria       HVAC_OverheadLine
1        Czech R      Austria       HVAC_OverheadLine
2        Germany      Austria       HVAC_OverheadLine
3        Hungary      Austria       HVAC_OverheadLine
4          Italy      Austria       HVAC_OverheadLine
..           ...          ...                     ...
635          NO4  Nordavind A  HVDC_OffshoreToOnshore
636          NO4  Nordavind D  HVDC_OffshoreToOnshore
637          NO4  Nordavind C  HVDC_OffshoreToOnshore
638          NO2  Vestavind E  HVDC_OffshoreToOnshore
639          NO2    Sørvest D  HVDC_OffshoreToOnshore

[640 rows x 3 columns]

#### Tab: GeneratorsOfNode

In [18]:
generators_of_node_tab_public = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="GeneratorsOfNode", header=2)
generators_of_node_tab_public["unique_id"] = generators_of_node_tab_public.apply(lambda row: "".join(row["Node"] + row["Generator"]), axis=1)
generators_of_node_tab_public

Node          Generator                  unique_id
0    Bosnia H  Liginite existing  Bosnia HLiginite existing
1    Bulgaria  Liginite existing  BulgariaLiginite existing
2     Czech R  Liginite existing   Czech RLiginite existing
3     Germany  Liginite existing   GermanyLiginite existing
4     Estonia  Liginite existing   EstoniaLiginite existing
..        ...                ...                        ...
737       NO1      Hydrogen CCGT           NO1Hydrogen CCGT
738       NO2      Hydrogen CCGT           NO2Hydrogen CCGT
739       NO3      Hydrogen CCGT           NO3Hydrogen CCGT
740       NO4      Hydrogen CCGT           NO4Hydrogen CCGT
741       NO5      Hydrogen CCGT           NO5Hydrogen CCGT

[742 rows x 3 columns]

In [20]:
additional_from_open = pd.read_excel("Data_OpenEMPIRE/Sets.xlsx", sheet_name="GeneratorsOfNode", header=2)
additional_from_open["unique_id"] = additional_from_open.apply(lambda row: "".join(row["Node"] + row["Generator"]), axis=1)
additional_from_open = additional_from_open[~additional_from_open["unique_id"].isin(generators_of_node_tab_public["unique_id"].values)]
additional_from_open

Node               Generator  \
604             Croatia                     Geo   
613            Bosnia H  Hydro run-of-the-river   
628   Helgolander Bucht  Wind offshore grounded   
629            Nordsoen  Wind offshore grounded   
631   Sorlige Nordsjo I  Wind offshore grounded   
632  Sorlige Nordsjo II  Wind offshore grounded   
656   Helgolander Bucht  Wind offshore floating   
657            Nordsoen  Wind offshore floating   
659   Sorlige Nordsjo I  Wind offshore floating   
660  Sorlige Nordsjo II  Wind offshore floating   
661             Austria                 Nuclear   
662            Bosnia H                 Nuclear   
668             Denmark                 Nuclear   
669             Estonia                 Nuclear   
674              Greece                 Nuclear   
675             Croatia                 Nuclear   
677             Ireland                 Nuclear   
678               Italy                 Nuclear   
680             Luxemb.                 Nuclear   
681              Latvia                 Nuclear   
682           Macedonia                 Nuclear   
685            Portugal                 Nuclear   
687              Serbia                 Nuclear   
691                 NO1                 Nuclear   
692                 NO2                 Nuclear   
693                 NO3                 Nuclear   
694                 NO4                 Nuclear   
695                 NO5                 Nuclear   

                                    unique_id  
604                                CroatiaGeo  
613            Bosnia HHydro run-of-the-river  
628   Helgolander BuchtWind offshore grounded  
629            NordsoenWind offshore grounded  
631   Sorlige Nordsjo IWind offshore grounded  
632  Sorlige Nordsjo IIWind offshore grounded  
656   Helgolander BuchtWind offshore floating  
657            NordsoenWind offshore floating  
659   Sorlige Nordsjo IWind offshore floating  
660  Sorlige Nordsjo IIWind offshore floating  
661                            AustriaNuclear  
662                           Bosnia HNuclear  
668                            DenmarkNuclear  
669                            EstoniaNuclear  
674                             GreeceNuclear  
675                            CroatiaNuclear  
677                            IrelandNuclear  
678                              ItalyNuclear  
680                            Luxemb.Nuclear  
681                             LatviaNuclear  
682                          MacedoniaNuclear  
685                           PortugalNuclear  
687                             SerbiaNuclear  
691                                NO1Nuclear  
692                                NO2Nuclear  
693                                NO3Nuclear  
694                                NO4Nuclear  
695                                NO5Nuclear

Remove offshore wind rows (incl. but different naming convention)

In [21]:
wo_values = ["Wind offshore grounded", "Wind offshore floating"]
additional_from_open = additional_from_open[~additional_from_open["Generator"].isin(wo_values)]
generators_of_node_tab = pd.concat([generators_of_node_tab_public, additional_from_open], ignore_index=True)
generators_of_node_tab = generators_of_node_tab.drop(columns=["unique_id"]).reset_index(drop=True)
generators_of_node_tab

Node          Generator
0    Bosnia H  Liginite existing
1    Bulgaria  Liginite existing
2     Czech R  Liginite existing
3     Germany  Liginite existing
4     Estonia  Liginite existing
..        ...                ...
757       NO1            Nuclear
758       NO2            Nuclear
759       NO3            Nuclear
760       NO4            Nuclear
761       NO5            Nuclear

[762 rows x 2 columns]

Remove and update offshore wind areas

In [22]:
generators_of_node_tab = generators_of_node_tab[~generators_of_node_tab["Node"].isin(NODES_TO_BE_REMOVED)]
generators_of_node_tab = generators_of_node_tab[~generators_of_node_tab["Node"].isin(offshr_areas)].reset_index(drop=True)
generators_of_node_tab

Node          Generator
0    Bosnia H  Liginite existing
1    Bulgaria  Liginite existing
2     Czech R  Liginite existing
3     Germany  Liginite existing
4     Estonia  Liginite existing
..        ...                ...
729       NO1            Nuclear
730       NO2            Nuclear
731       NO3            Nuclear
732       NO4            Nuclear
733       NO5            Nuclear

[734 rows x 2 columns]

**NOTE: For Scenario generation EMPIRE needs both techs to be included for all offshore wind nodes!**

In [31]:
grounded_areas = pd.read_csv("Output/offshore_capacities_base.csv")[["Node"]].drop_duplicates(subset='Node', keep="first", ignore_index=True)
grounded_areas["Generator"] = "Wind offshore grounded"
floating_areas = grounded_areas.copy()
floating_areas["Generator"] = "Wind offshore floating"
floating_areas

Node               Generator
0         Moray Firth  Wind offshore floating
1      Firth of Forth  Wind offshore floating
2         Dogger Bank  Wind offshore floating
3             Hornsea  Wind offshore floating
4       Outer Dowsing  Wind offshore floating
5             Norfolk  Wind offshore floating
6         East Anglia  Wind offshore floating
7            Borssele  Wind offshore floating
8     Hollandsee Kust  Wind offshore floating
9   Helgoländer Bucht  Wind offshore floating
10           Nordsøen  Wind offshore floating
11         Nordvest A  Wind offshore floating
12         Nordvest C  Wind offshore floating
13        Vestavind A  Wind offshore floating
14        Sønnavind A  Wind offshore floating
15          Sørvest C  Wind offshore floating
16         Nordvest B  Wind offshore floating
17        Vestavind F  Wind offshore floating
18        Vestavind B  Wind offshore floating
19        Vestavind C  Wind offshore floating
20        Vestavind D  Wind offshore floating
21          Sørvest F  Wind offshore floating
22          Sørvest B  Wind offshore floating
23        Nordavind B  Wind offshore floating
24        Nordavind A  Wind offshore floating
25        Nordavind D  Wind offshore floating
26        Nordavind C  Wind offshore floating
27        Vestavind E  Wind offshore floating
28          Sørvest E  Wind offshore floating
29          Sørvest A  Wind offshore floating
30          Sørvest D  Wind offshore floating

In [32]:
generators_of_node_tab = pd.concat([generators_of_node_tab, grounded_areas, floating_areas], ignore_index=True)
generators_of_node_tab

Node               Generator
0       Bosnia H       Liginite existing
1       Bulgaria       Liginite existing
2        Czech R       Liginite existing
3        Germany       Liginite existing
4        Estonia       Liginite existing
..           ...                     ...
791  Nordavind C  Wind offshore floating
792  Vestavind E  Wind offshore floating
793    Sørvest E  Wind offshore floating
794    Sørvest A  Wind offshore floating
795    Sørvest D  Wind offshore floating

[796 rows x 2 columns]

#### Tab: GeneratorsOfTechnology

Keep as is

### Write to Excel and update sheets

Sheets to be updated and corresponding dataframe

In [33]:
SHEET_TO_DF_AND_ROW = dict({
    "Nodes": [nodes_tab, 2],
    "DirectionalLines": [directional_lines_tab, 4],
    "LineTypeOfDirectionalLines": [line_type_of_directional_lines_tab, 4],
    "GeneratorsOfNode": [generators_of_node_tab, 4]
})

In [34]:
# Clean and overwrite existing input_file from original
import shutil

source = "Data_EMPIRE-Public/Sets.xlsx"
destination = "EMPIRE_input/Sets.xlsx"

shutil.copy(source, destination)

'EMPIRE_input/Sets.xlsx'

In [35]:
import openpyxl as ox

def update_spreadsheet(path:str, sheet_name:str, _df : pd.DataFrame, startcol:int=1, startrow:int=4):
    wb = ox.load_workbook(path)
    ws=wb[sheet_name]
    for row in range(0, _df.shape[0]): #For each row in the dataframe
        for col in range(0, _df.shape[1]): #For each column in the dataframe
            ws.cell(row = startrow + row, column = startcol + col).value = _df.iat[row, col]
    wb.save(path)

In [36]:
#Call the defined function
for sheet, df_and_row in SHEET_TO_DF_AND_ROW.items():
    df = df_and_row[0]
    row = df_and_row[1]
    update_spreadsheet(destination, sheet, df, startrow=row)